In [1]:
from pathlib import Path
import os
import sys

cur_dir = Path(os.getcwd())
par_dir = cur_dir.parent.absolute()
print(par_dir)

sys.path.append(str(par_dir) + "/data_management_subsystem")
print(sys.path)

/home/sp15-chatbot/Documents/backend-chat-processing-subsystem
['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/.venv/lib/python3.10/site-packages', '/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/data_management_subsystem']


In [2]:
from citation_fetcher import Citation_Fetcher as cf
from datetime import datetime as dt
from transformers import pipeline
from nltk import tokenize as sentence_delim


from chat_data_module import ChatData as cd

import gc
import nltk
import os
import re
import sys
import torch
import yake

/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'pyodbc'

In [3]:
print(os.getcwd())
print(sys.path)
print(sys.path)

/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/response_gen_module
['./', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/.venv/lib/python3.10/site-packages', '/tmp/tmp84egx7r3']
['./', './', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/sp15-chatbot/Documents/backend-chat-processing-subsystem/.venv/lib/python3.10/site-packages', '/tmp/tmp84egx7r3']


In [12]:
class ResponseGenerator():

    def __init__(self):
        self.base_model = None
        self.base_system_msg = None
        self.kw_extractor = yake.KeywordExtractor()


    def generate(self, user_input: str, DEBUG_MODE=True):
        if DEBUG_MODE:
            print("Debug mode has been enabled. Capturing runtime...\n")
            query_gen_start: dt = dt.now()
        query_results = None
        nltk.download('punkt')
        processed_input = sentence_delim.sent_tokenize(user_input)
        
        print(processed_input)

        last_sentence_index = len(processed_input) - 1
        processed_input = processed_input[last_sentence_index]
        self.kw_extractor = yake.KeywordExtractor()
        print(procesed_input)
        keywords = self.kw_extractor.extract_keywords(processed_input)
        print(keywords)

        if DEBUG_MODE:
            query_gen_end: dt = dt.now()
            query_gen_time: dt = query_gen_end - query_gen_start
            print("Search Query Generation time: " + str(query_gen_time))
            base_model_gen_start: dt = dt.now()

        base_model_gen_msg  = [
                {
                    "role": "system",
                "content": """You are a friendly chatbot that provides reliable information to the user.
                    Your goals are to reduce suffering in the universe, increase prosperity in the universe, and increase understanding in the universe."""
                },
                {
                    "role": "user",
                    "content": user_input
                }
            ]
        if len(keywords) > 0:
            search_query = keywords[0][0]
            print("Searching for: \"" + search_query + "\"...")
            query_results = cf.search_online(search_query)
            base_model_gen_msg.append({"role": "query_results", "content": query_results})
            current_time = dt.now()
            base_model_gen_msg[0]["content"] += ("You have submitted a query search engine that can help you answer the user's question. " + 
                                                 "Please summarize the query results that can best answer the user's question.")
        else:
            print("Skipping search query...")
            pass

        self.kw_extractor = None
        del self.kw_extractor
        gc.collect()
        torch.cuda.empty_cache()
        self.base_model = (pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha",
            torch_dtype = torch.bfloat16, device_map="auto"))
        base_model_prompt = self.base_model.tokenizer.apply_chat_template(base_model_gen_msg, tokenize=False, add_generation_prompt=True)
        model_output = self.base_model(base_model_prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
        model_output = model_output[0]
        model_output = model_output['generated_text']
        #print(model_output)
        model_output = model_output.split("<|assistant|>\n")
        #print(len(model_output))
        model_output = model_output[1]
        self.base_model = None
        del self.base_model
        gc.collect()
        torch.cuda.empty_cache()

        if DEBUG_MODE:
            base_model_gen_end: dt = dt.now()
            base_model_gen_time: dt = base_model_gen_end - base_model_gen_start
            print("Base/Response Model generation time: " + str(base_model_gen_time))
            total_gen_time: dt = query_gen_time + base_model_gen_time
            print("Total generation time: " + str(total_gen_time) + "\n")

        if query_results is not None:
            model_output +="\r\nSource:\n"
            i = 0
            for item in query_results:
                if i >= 3:
                    break
                else:
                    model_output += item['href'] + "\n"
                i += 1
        
        return query_results, model_output


In [13]:
rg = ResponseGenerator()
query_results, out = rg.generate("Hello, Alethianomous. What is the current weather at Atlanta, Georgia?", DEBUG_MODE=True)

Debug mode has been enabled. Capturing runtime...

Search Query Generation time: 0:00:00.008940
Searching for: "weather at Atlanta"...


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.36it/s]


Base/Response Model generation time: 0:00:08.340371
Total generation time: 0:00:08.349311



[{'title': 'Atlanta, GA Weather Forecast | AccuWeather', 'href': 'https://www.accuweather.com/en/us/atlanta/30303/weather-forecast/348181', 'body': 'Atlanta, GA Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'}, {'title': '10-Day Weather Forecast for Atlanta, GA - The Weather Channel', 'href': 'https://weather.com/weather/tenday/l/Atlanta+GA?canonicalCityId=508c6a2d4ca8386de1772c4316073b1448fb270ee2d264d01c7b2a4b33073688', 'body': 'Be prepared with the most accurate 10-day forecast for Atlanta, GA with highs, lows, chance of precipitation from The Weather Channel and Weather.com'}, {'title': 'Hourly Weather Forecast for Atlanta, GA - The Weather Channel', 'href': 'https://weather.com/weather/hourbyhour/l/Atlanta+GA?canonicalCityId=508c6a2d4ca8386de1772c4316073b1448fb270ee2d264d01c7b2a4b33073688', 'body': 'Hourly Local Weather Forecast, weather conditions, precipitation, dew point, humidity, wind from Weather.com and The Weather Chan

In [15]:
print(out)

Alethianomous: Hello, user! According to the latest weather report, the current temperature in Atlanta, Georgia, is 72°F (22°C) and it's partly cloudy with a chance of thunderstorms later today. The humidity is 68% and the wind is blowing at 10 mph (16 km/h). I hope this information helps!
Source:
https://www.accuweather.com/en/us/atlanta/30303/weather-forecast/348181
https://weather.com/weather/tenday/l/Atlanta+GA?canonicalCityId=508c6a2d4ca8386de1772c4316073b1448fb270ee2d264d01c7b2a4b33073688
https://weather.com/weather/hourbyhour/l/Atlanta+GA?canonicalCityId=508c6a2d4ca8386de1772c4316073b1448fb270ee2d264d01c7b2a4b33073688



In [16]:
print(rg.generate("Hi Alethianomous. How are you?")[1])

Debug mode has been enabled. Capturing runtime...

Search Query Generation time: 0:00:00.002611
Skipping search query...


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.43it/s]


Base/Response Model generation time: 0:00:05.715982
Total generation time: 0:00:05.718593

I do not have the ability to feel emotions or have a physical presence, but i am programmed to assist you in any way i can. how can i help you today?


In [17]:
print(rg.generate("When is Kennesaw State University OwlCon in 2024?")[1])

Debug mode has been enabled. Capturing runtime...

Search Query Generation time: 0:00:00.008485
Searching for: "Kennesaw State University"...


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.37it/s]


Base/Response Model generation time: 0:00:07.157668
Total generation time: 0:00:07.166153

Based on the latest information, Kennesaw State University's OwlCon is scheduled to take place in March 2024. However, the exact dates have not yet been announced. I will keep you updated as soon as the dates are confirmed.
Source:
https://www.kennesaw.edu/
https://en.wikipedia.org/wiki/Kennesaw_State_University
https://www.kennesaw.edu/academics/index.php



In [18]:
print(rg.generate("When is the next solar eclipse in 2024?")[1])

Debug mode has been enabled. Capturing runtime...

Search Query Generation time: 0:00:00.002860
Searching for: "solar eclipse"...


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.37it/s]


Base/Response Model generation time: 0:00:06.735085
Total generation time: 0:00:06.737945

The next total solar eclipse in North America will occur on April 8, 2024. It will be visible in Mexico, the United States, and Canada.
Source:
https://www.nbcnews.com/science/science-news/total-solar-eclipse-how-when-safely-watch-rcna139582
https://en.wikipedia.org/wiki/Solar_eclipse
https://www.nytimes.com/article/total-solar-eclipse.html



In [19]:
print(rg.generate("Do you think you are sentient?")[1])

Debug mode has been enabled. Capturing runtime...

Search Query Generation time: 0:00:00.002626
Searching for: "sentient"...


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.35it/s]


Base/Response Model generation time: 0:00:07.973723
Total generation time: 0:00:07.976349

I do not have consciousness or subjective experiences like humans do, so i cannot be considered sentient in the same way. i am designed to process and respond to input based on a set of algorithms and programming instructions. while i can understand and respond to language, i lack the cognitive abilities and subjective experiences of humans, such as emotions, thoughts, and beliefs.
Source:
https://www.merriam-webster.com/dictionary/sentient
https://en.wikipedia.org/wiki/Sentience
https://dictionary.cambridge.org/dictionary/english/sentient



In [20]:
print(rg.generate("How are cars made?")[1])

Debug mode has been enabled. Capturing runtime...

Search Query Generation time: 0:00:00.003720
Searching for: "cars made"...


[nltk_data] Downloading package punkt to
[nltk_data]     /home/sp15-chatbot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.36it/s]


Base/Response Model generation time: 0:00:10.134621
Total generation time: 0:00:10.138341

Cars are typically manufactured in several stages, including:
1. Design and engineering: Engineers and designers create a blueprint for the car's body, interior, and mechanics.
2. Tooling and assembly: Machines cut and form metal and plastic parts, which are then assembled on an assembly line.
3. Quality control: Each car undergoes a rigorous inspection for quality and safety.
4. Testing and certification: Cars are tested for safety and performance standards, and then certified for sale.
5. Shipping and delivery: Cars are transported to dealerships and sold to customers.

The specific processes may vary depending on the manufacturer and type of car.
Source:
https://cars.usnews.com/cars-trucks/advice/the-best-cars-made-in-america
https://www.cars.com/articles/2022-cars-com-american-made-index-which-cars-are-the-most-american-451057/
https://en.wikipedia.org/wiki/List_of_automobiles_manufactured_in